# Homework 4: Modeling Text Data

### Team Member 1:
* UNI:  WL2522
* Name: Wilson Lui

### Team Member 2 [optional]:
* UNI:  
* Name:


You can find the data here: https://data.boston.gov/dataset/vision-zero-entry

# Task1 - Data Cleaning  [10 points]

Load the data, visualize the class distribution. Clean up the target labels. Some categories have been arbitrarily split and need to be consolidated. 

Having removed the embedded image tags from the categories, the following categories are consolidated due to them having the same meaning with different wordings:


1) "bike facilities don't exist or need improvement" / "there are no bike facilities or they need maintenance"

2) "sidewalks/ramps don't exist or need improvement" / "there are no sidewalks or they need maintenance"

3) "the wait for the "Walk" signal is too long" / "people have to wait too long for the "Walk" signal"

4) "the roadway surface needs improvement" / "the roadway surface needs maintenance"

5) "it’s hard to see / low visibility" / "it’s hard for people to see each other"

The following categories are very similar but not quite the same, and thus were not combined during this preprocessing step:


"people don't yield while going straight" / "people don't yield while turning" /  "people run red lights / stop signs"


Whenever categories are consolidated, I merge the category with fewer data points into the one with more data points.


In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split

data = pd.read_csv('Vision_Zero_Entry.csv')


In [2]:
#Remove rows with no comments
#Create a list of categories
#Consolidate the categories that have embedded images tags into corresponding categories


data = data[data['COMMENTS'].notnull()]

print(len(data['REQUESTTYPE'].unique()))
categories = data['REQUESTTYPE'].unique()
categories = categories.tolist()
categories.sort()

for a in range(len(categories)): print(categories[a])
    
data['REQUESTTYPE'].replace(to_replace=categories[0:7], value=[
        "there's not enough time to cross the street",
    'the wait for the "Walk" signal is too long',
    'people speed', 'it’s hard to see / low visibility',
        "sidewalks/ramps don't exist or need improvement", "the roadway surface needs improvement",
    "of something that is not listed here"], inplace=True)

print(data['REQUESTTYPE'].unique())
print(len(data['REQUESTTYPE'].unique()))


28
" src="images/01 - Not enough time to cross.png"></span>&nbsp;there's not enough time to cross the street
" src="images/02 - Wait is too long.png"></span>&nbsp;the wait for the "Walk" signal is too long
" src="images/06 - Speeding.png"></span>&nbsp;people speed
" src="images/10 - Hard to see.png"></span>&nbsp;it’s hard to see / low visibility
" src="images/11 - Sidewalk issue.png"></span>&nbsp;sidewalks/ramps don't exist or need improvement
" src="images/12 - Bike facility issue.png"></span>&nbsp;the roadway surface needs improvement
" src="images/14 - Other issue.png"></span>&nbsp;of something that is not listed here
bike facilities don't exist or need improvement
it's too far / too many lanes to cross
it’s hard for people to see each other
it’s hard to see / low visibility
of something that is not listed here
people are not given enough time to cross the street
people cross away from the crosswalks
people don't yield while going straight
people don't yield while turning
people dou

In [3]:
#Count how many occurrences of each category occur in the dataset
#Create a list with tuples containing each category and its number of occurrences


count = Counter(data['REQUESTTYPE'])
count = count.items()
count = list(count)
count.sort()
print(count)

print(count[14][0], count[19][0])
print(count[0][0], count[18][0])
print(count[11][0], count[17][0])
print(count[15][0], count[16][0])
print(count[2][0], count[3][0])


[("bike facilities don't exist or need improvement", 782), ("it's too far / too many lanes to cross", 84), ('it’s hard for people to see each other', 28), ('it’s hard to see / low visibility', 391), ('of something that is not listed here', 1425), ('people are not given enough time to cross the street', 9), ('people cross away from the crosswalks', 255), ("people don't yield while going straight", 264), ("people don't yield while turning", 461), ('people double park their vehicles', 426), ('people have to cross too many lanes / too far', 27), ('people have to wait too long for the "Walk" signal', 30), ('people run red lights / stop signs', 662), ('people speed', 746), ("sidewalks/ramps don't exist or need improvement", 303), ('the roadway surface needs improvement', 222), ('the roadway surface needs maintenance', 34), ('the wait for the "Walk" signal is too long', 208), ('there are no bike facilities or they need maintenance', 128), ('there are no sidewalks or they need maintenance', 40

In [4]:
#For each category pair, check which category appears less often
#Replace that category with the other category


#bike facilities don't exist or need improvement / there are no bike facilities or they need maintenance


if count[0][1] > count[18][1]: 
    data['REQUESTTYPE'].replace(to_replace=count[18][0], value=count[0][0], inplace=True)

elif count[18][1] > count[0][1]:
    data['REQUESTTYPE'].replace(to_replace=count[0][0], value=count[18][0], inplace=True)

    
#sidewalks/ramps don't exist or need improvement / there are no sidewalks or they need maintenance


if count[14][1] > count[19][1]:
    data['REQUESTTYPE'].replace(to_replace=count[19][0], value=count[14][0], inplace=True)
    
elif count[19][1] > count[14][1]:
    data['REQUESTTYPE'].replace(to_replace=count[14][0], value=count[19][0], inplace=True)
    
    
#people have to wait too long for the "Walk" signal / the wait for the "Walk" signal is too long


if count[11][1] > count[17][1]:
    data['REQUESTTYPE'].replace(to_replace=count[17][0], value=count[11][0], inplace=True)
    
elif count[17][1] > count[11][1]:
    data['REQUESTTYPE'].replace(to_replace=count[11][0], value=count[17][0], inplace=True)
    

#the roadway surface needs improvement / the roadway surface needs maintenance


if count[15][1] > count[16][1]:
    data['REQUESTTYPE'].replace(to_replace=count[16][0], value=count[15][0], inplace=True)
    
elif count[16][1] > count[15][1]:
    data['REQUESTTYPE'].replace(to_replace=count[15][0], value=count[16][0], inplace=True)
    
    
#it’s hard to see / low visibility / it’s hard for people to see each other


if count[2][1] > count[3][1]:
    data['REQUESTTYPE'].replace(to_replace=count[3][0], value=count[2][0], inplace=True)
    
elif count[3][1] > count[2][1]:
    data['REQUESTTYPE'].replace(to_replace=count[2][0], value=count[3][0], inplace=True)


print(Counter(data['REQUESTTYPE']))
print(len(data['REQUESTTYPE'].unique()))


Counter({'of something that is not listed here': 1425, "bike facilities don't exist or need improvement": 910, 'people speed': 746, 'people run red lights / stop signs': 662, "people don't yield while turning": 461, 'people double park their vehicles': 426, 'it’s hard to see / low visibility': 419, "sidewalks/ramps don't exist or need improvement": 343, "people don't yield while going straight": 264, 'the roadway surface needs improvement': 256, 'people cross away from the crosswalks': 255, 'the wait for the "Walk" signal is too long': 238, "there's not enough time to cross the street": 122, "it's too far / too many lanes to cross": 84, 'people have to cross too many lanes / too far': 27, 'people are not given enough time to cross the street': 9})
16


In [5]:
#Separate the features from the response variable


target = data['REQUESTTYPE']
comments = data['COMMENTS']


# Task2 - Model 1 [10 points]

Run a baseline multi-class classification model using a bag-of-word approach, report macro f1-score (should be above .5) and visualize the confusion matrix. Can you interpret the mistakes made by the model? 

In [6]:
vect = CountVectorizer()
lr = LogisticRegressionCV()

comments_train, comments_test, target_train, target_test = train_test_split(comments, target, stratify=target)
print(type(comments_train))
np.savetxt('comments.csv', comments, delimiter=',')
print(type(comments_train[7000]))
X_train = vect.fit_transform(comments_train)
X_test = vect.transform(comments_test)


print(vect.score(X_test, y_test))



<class 'pandas.core.series.Series'>


TypeError: Mismatch between array dtype ('object') and format specifier ('%.18e')

# Task3 - Model 2 [30 points]

Improve the model using more complex text features, including n-grams, character n-grams and possibly domain-specific features.

In [3]:
# Add your code for task 3 here. You may use multiple cells. 



# Task4 - Visualize Results [10 points]

Visualize results of the tuned model (classification results, confusion matrix, important features, example mistakes).

In [4]:
# Add your code for task 4 here. You may use multiple cells. 



# Task5 - Clustering [10 points]

Apply LDA, NMF and K-Means to the whole dataset. Can you find clusters or topics that match well with some of the ground truth labels? Use ARI to compare the methods and visualize topics and clusters.

In [5]:
# Add your code for task 5 here. You may use multiple cells. 



# Task6 - Model 3 [30 points]

Improve the class definition for REQUESTTYPE by using the results of the clustering and results of the previous classification model. Re-assign labels using either the results of clustering or using keywords that you found during data exploration. The labels must be semantically meaningful.
The data has a large “other” category. Apply the topic modeling and clustering techniques to this subset of the data to find possible splits of this class.
Report accuracy using macro average f1 score (should be above .53) 


In [6]:
# Add your code for task 6 here. You may use multiple cells. 



# Extra Credit [Up to +20 points]

Use a word embedding representation like word2vec for step 3 and or step 6. 

In [7]:
# Add your code for extra credit here. You may use multiple cells. 

